<a href="https://colab.research.google.com/github/EMSIMa/ADD3IIR/blob/main/10_Generateurs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Générateurs

Nous allons ici approfondir notre connaissance des générateurs Python, y compris les *generator expressions * et les *fonctions génératrices*.

## Generator Expressions

La différence entre les compréhensions de listes et les expressions de générateurs peut parfois prêter à confusion :

### Les compréhensions de listes utilisent des crochets, tandis que les expressions de générateurs utilisent des parenthèses.
Il s'agit d'une compréhension de liste représentative :

In [ ]:
[n ** 2 for n in range(12)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]

Il s'agit d'une expression représentative du générateur :

In [ ]:
(n ** 2 for n in range(12))

<generator object <genexpr> at 0x104a60518>

Notez que si on affiche l'expression du générateur, on n'affiche pas son contenu ; une façon d'afficher le contenu d'une expression du générateur est de la passer au constructeur ``list`` :

In [ ]:
G = (n ** 2 for n in range(12))
list(G)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]

### Une liste est une collection de valeurs, tandis qu'un générateur est une recette pour produire des valeurs.
Lorsque vous créez une liste, vous construisez en fait une collection de valeurs, et il y a un certain coût de mémoire associé à cela.
Lorsque vous créez un générateur, vous ne construisez pas une collection de valeurs, mais une recette pour produire ces valeurs.
Les deux exposent la même interface d'itérateur, comme nous pouvons le voir ici :

In [ ]:
L = [n ** 2 for n in range(12)]
for val in L:
    print(val, end=' ')

0 1 4 9 16 25 36 49 64 81 100 121 

In [ ]:
G = (n ** 2 for n in range(12))
for val in G:
    print(val, end=' ')

0 1 4 9 16 25 36 49 64 81 100 121 

La différence réside dans le fait qu'une expression génératrice ne calcule les valeurs qu'au moment où elles sont nécessaires.
Cela permet non seulement d'économiser de la mémoire, mais aussi d'améliorer l'efficacité des calculs !
Cela signifie également que la taille d'une liste est limitée par la mémoire disponible, alors que la taille d'une expression de générateur est illimitée !

Un exemple d'expression de générateur infini peut être créé en utilisant l'itérateur ``count`` défini dans ``itertools`` :

In [ ]:
from itertools import count
count()

count(0)

In [ ]:
for i in count():
    print(i, end=' ')
    if i >= 10: break

0 1 2 3 4 5 6 7 8 9 10 

L'itérateur ``count`` continuera à compter joyeusement pour toujours jusqu'à ce que vous lui disiez d'arrêter ; cela rend pratique la création de générateurs qui continueront à compter pour toujours :

In [ ]:
factors = [2, 3, 5, 7]
G = (i for i in count() if all(i % n > 0 for n in factors))
for val in G:
    print(val, end=' ')
    if val > 40: break

1 11 13 17 19 23 29 31 37 41 

Vous voyez peut-être où nous voulons en venir : si nous allongions la liste des facteurs de manière appropriée, nous aurions les prémices d'un générateur de nombres premiers, utilisant l'algorithme du crible d'Eratosthène. Nous examinerons ce point plus en détail dans un instant.

### Une liste peut être itérée plusieurs fois ; une expression de générateur est à usage unique
C'est l'un des problèmes potentiels des expressions de générateur.
Avec une liste, nous pouvons simplement faire ceci :

In [ ]:
L = [n ** 2 for n in range(12)]
for val in L:
    print(val, end=' ')
print()

for val in L:
    print(val, end=' ')

0 1 4 9 16 25 36 49 64 81 100 121 
0 1 4 9 16 25 36 49 64 81 100 121 

Une expression génératrice, en revanche, est épuisée après une itération :

In [ ]:
G = (n ** 2 for n in range(12))
list(G)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]

In [ ]:
list(G)

[]

Cela peut être très utile car cela signifie que l'itération peut être arrêtée et démarrée :

In [ ]:
G = (n**2 for n in range(12))
for n in G:
    print(n, end=' ')
    if n > 30: break

print("\ndoing something in between")

for n in G:
    print(n, end=' ')

0 1 4 9 16 25 36 
doing something in between
49 64 81 100 121 

C'est notamment utile lorsque l'on travaille avec des collections de fichiers de données sur disque

cela signifie que vous pouvez facilement les analyser par lots, en laissant le générateur suivre ceux que vous n'avez pas encore vus.

## Fonctions génératrices : Utilisation de ``yield``
Nous avons vu dans la section précédente que les compréhensions de listes sont mieux utilisées pour créer des listes relativement simples, alors que l'utilisation d'une boucle ``for`` normale peut être meilleure dans des situations plus compliquées.
Il en va de même pour les expressions de générateur : nous pouvons créer des générateurs plus compliqués en utilisant les *fonctions génératrices*, qui utilisent l'instruction ``yield``.

Ici, nous avons deux façons de construire la même liste :

In [ ]:
L1 = [n ** 2 for n in range(12)]

L2 = []
for n in range(12):
    L2.append(n ** 2)

print(L1)
print(L2)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]
[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121]


De même, nous avons ici deux façons de construire des générateurs équivalents :

In [ ]:
G1 = (n ** 2 for n in range(12))

def gen():
    for n in range(12):
        yield n ** 2

G2 = gen()
print(*G1)
print(*G2)

0 1 4 9 16 25 36 49 64 81 100 121
0 1 4 9 16 25 36 49 64 81 100 121


Une fonction génératrice est une fonction qui, au lieu d'utiliser ``return`` pour retourner une valeur une seule fois, utilise ``yield`` pour retourner une séquence (potentiellement infinie) de valeurs.
Tout comme dans les expressions de générateur, l'état du générateur est préservé entre les itérations partielles, mais si nous voulons une nouvelle copie du générateur, nous pouvons simplement appeler la fonction à nouveau.

## Exemple : Générateur de nombres premiers
Voici un exemple de fonction génératrice : une fonction permettant de générer une série illimitée de nombres premiers.
Un algorithme classique pour cela est le *crible d'Eratosthène*, qui fonctionne comme suit :

In [ ]:
# Generate a list of candidates
L = [n for n in range(2, 40)]
print(L)

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]


In [ ]:
# Remove all multiples of the first value
L = [n for n in L if n == L[0] or n % L[0] > 0]
print(L)

[2, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39]


In [ ]:
# Remove all multiples of the second value
L = [n for n in L if n == L[1] or n % L[1] > 0]
print(L)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 25, 29, 31, 35, 37]


In [ ]:
# Remove all multiples of the third value
L = [n for n in L if n == L[2] or n % L[2] > 0]
print(L)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37]


Si nous répétons cette procédure suffisamment de fois sur une liste suffisamment grande, nous pouvons générer autant de nombres premiers que nous le souhaitons.

Encapsulons cette logique dans une fonction génératrice :

In [ ]:
def gen_primes(N):
    """Generate primes up to N"""
    primes = set()
    for n in range(2, N):
        if all(n % p > 0 for p in primes):
            primes.add(n)
            yield n

print(*gen_primes(100))

2 3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97
